In [ ]:
pip install flair

In [1]:
import pandas as pd

In [2]:
!gdown --i 1sRELpaE3GICTBvm9Pb9_cTc6y4GfTovt

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1sRELpaE3GICTBvm9Pb9_cTc6y4GfTovt
To: /content/full_data_125k.csv
100% 82.9M/82.9M [00:01<00:00, 68.9MB/s]


In [2]:
data = pd.read_csv('/content/full_data_125k.csv')
data

,Unnamed: 0,sentence,word_labels
0,0,This includes reviewing large numbers of docum...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In..."
1,1,Will report directly to Head of Weather Resear...,"O,O,O,O,O,O,O,In-SK,O,O,O,O,O"
2,2,The analyst will be expected to interpret resu...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In..."
3,3,A wide degree of creativity and latitude is ex...,"O,O,O,O,In-SK,O,O,O,O"
4,4,You must be authorized to work in the United S...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In-SK,O,..."
...,...,...,...
124399,124399,Simpson Strong Tie s drug policy is based upon...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
124400,124400,We believe all employees affect customers sal...,"O,O,O,O,O,O,In-SK,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
124401,124401,In keeping with the Company s commitment to th...,"O,O,O,O,O,O,O,O,O,O,O,O,In-SK,O,O,O,O,O,O,O,O"
124402,124402,Complete training as required to ensure you ar...,"O,In-SK,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"


# Since it takes a long time to train I use portion of data to train this

# on T4 GPU training takes about 4h

In [3]:
from tqdm import tqdm

with open("train.txt", 'w') as f:

  for j in tqdm(range(0,40000)):
    try:
      s = data['sentence'][j].split()
      l = data['word_labels'][j].split(',')
      for i in range(len(s)):
        text = "\n" + s[i] + ' '+ l[i]
        f.write(text)
      f.write('\n')
    except:
      pass


with open("test.txt", 'w') as f:

  for j in tqdm(range(40000,48000)):
    try:
      s = data['sentence'][j].split()
      l = data['word_labels'][j].split(',')
      for i in range(len(s)):
        text = "\n" + s[i] + ' '+ l[i]
        f.write(text)
      f.write('\n')
    except:
      pass


with open("dev.txt", 'w') as f:

  for j in tqdm(range(50000,55000)):
    try:
      s = data['sentence'][j].split()
      l = data['word_labels'][j].split(',')
      for i in range(len(s)):
        text = "\n" + s[i] + ' '+ l[i]
        f.write(text)
      f.write('\n')
    except:
      pass



100%|██████████| 5000/5000 [00:00<00:00, 7148.28it/s]


In [4]:
from flair.data import Sentence, Corpus
from flair.datasets import ColumnCorpus
from flair.models import SequenceTagger


# define columns
columns = {0 : 'text', 1 : 'ner'}
# directory where the data resides
data_folder = '.'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt',
                              test_file = 'test.txt',
                              dev_file = 'dev.txt')

2023-10-19 20:04:11,879 Reading data from .
2023-10-19 20:04:11,880 Train: train.txt
2023-10-19 20:04:11,883 Dev: dev.txt
2023-10-19 20:04:11,886 Test: test.txt


In [5]:
# tag to predict
tag_type = 'ner'
# make tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

<ipython-input-5-827ff065dbcf>:4: DeprecationWarning: Call to deprecated method make_tag_dictionary. (Use 'make_label_dictionary' instead.) -- Deprecated since version 0.8.
  tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)


In [6]:
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings
from typing import List
from flair.embeddings import TransformerWordEmbeddings


# init embedding
embedding = TransformerWordEmbeddings('distilbert-base-uncased')
# embeddings : StackedEmbeddings = StackedEmbeddings(
#                                  embeddings=embedding)

In [7]:
from flair.models import SequenceTagger
tagger : SequenceTagger = SequenceTagger(hidden_size=128,
                                       embeddings=embedding,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)

2023-10-19 20:05:31,841 SequenceTagger predicts: Dictionary with 4 tags: O, In-SK, <START>, <STOP>
SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30523, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_

In [8]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train('flair-ner',
              learning_rate=0.04,
              mini_batch_size=16,
              max_epochs=150, save_model_each_k_epochs=1)

2023-10-19 20:05:46,971 ----------------------------------------------------------------------------------------------------
2023-10-19 20:05:46,973 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30523, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_fea

100%|██████████| 313/313 [02:49<00:00,  1.85it/s]

2023-10-19 20:40:50,216 Evaluating as a multi-label problem: False


2023-10-19 20:40:50,447 DEV : loss 0.024040572345256805 - f1-score (micro avg)  0.9629
2023-10-19 20:40:51,725 BAD EPOCHS (no improvement): 0
2023-10-19 20:40:51,731 saving best model
2023-10-19 20:40:56,019 ----------------------------------------------------------------------------------------------------
2023-10-19 20:44:35,350 epoch 2 - iter 250/2500 - loss 0.01772449 - time (sec): 219.32 - samples/sec: 1343.13 - lr: 0.040000
2023-10-19 20:48:23,308 epoch 2 - iter 500/2500 - loss 0.01760430 - time (sec): 447.28 - samples/sec: 1344.37 - lr: 0.040000
2023-10-19 20:52:13,378 epoch 2 - iter 750/2500 - loss 0.01644673 - time (sec): 677.35 - samples/sec: 1335.81 - lr: 0.040000
2023-10-19 20:55:58,814 epoch 2 - iter 1000/2500 - loss 0.01615531 - time (sec): 902.79 - samples/sec: 1339.86 - lr: 0.040000
2023-10-19 20:59:42,249 epoch 2 - iter 1250/2500 - loss 0.01588678 - time (sec): 1126.22 - samples/sec: 1340.52 - lr: 0.040000
2023-10-19 21:03:28,499 epoch 2 - iter 1500/2500 - loss 0.01521

100%|██████████| 313/313 [02:46<00:00,  1.88it/s]

...

2023-10-19 21:21:03,254 DEV : loss 0.009646009653806686 - f1-score (micro avg)  0.9834
2023-10-19 21:21:05,484 BAD EPOCHS (no improvement): 0
2023-10-19 21:21:05,497 saving best model
2023-10-19 21:21:08,378 ----------------------------------------------------------------------------------------------------
2023-10-19 21:21:28,754 ----------------------------------------------------------------------------------------------------
2023-10-19 21:21:28,757 Exiting from training early.
2023-10-19 21:21:28,759 Saving model ...
2023-10-19 21:21:29,493 Done.
2023-10-19 21:21:29,499 ----------------------------------------------------------------------------------------------------
2023-10-19 21:21:32,265 SequenceTagger predicts: Dictionary with 4 tags: O, In-SK, <START>, <STOP>


100%|██████████| 500/500 [03:18<00:00,  2.52it/s]

2023-10-19 21:24:50,698 Evaluating as a multi-label problem: False


2023-10-19 21:24:50,992 0.9827	0.9847	0.9837	0.9679
2023-10-19 21:24:50,993 
Results:
- F-score (micro) 0.9837
- F-score (macro) 0.9837
- Accuracy 0.9679

By class:
              precision    recall  f1-score   support

       In-SK     0.9827    0.9847    0.9837     40230

   micro avg     0.9827    0.9847    0.9837     40230
   macro avg     0.9827    0.9847    0.9837     40230
weighted avg     0.9827    0.9847    0.9837     40230

2023-10-19 21:24:50,996 ----------------------------------------------------------------------------------------------------


{'test_score': 0.9836985536035755,
 'dev_score_history': [0.9628695309420576, 0.9834273555993674],
 'train_loss_history': [0.04430018047908408, 0.01416487145773703],
 'dev_loss_history': [0.024040572345256805, 0.009646009653806686]}